# Stage 1: Extraction

### Было написано 40 кейсов для разработки и тестирования агента

In [1]:
with open("../data/user_messages.txt", "r", encoding='UTF-8') as file:
    message_samples = [line.rstrip() for line in file]

In [2]:
print(message_samples[0])

Какие книги стоит прочитать каждому? - Эта тема заинтересует любителей литературы, стремящихся расширить свой кругозор и найти вдохновение. Публикация в социальной сети поможет привлечь внимание к книгам, которые могут изменить жизнь читателя. ВК


In [3]:
import os

def get_models(directory, extension=".gguf"):
    return [filename.split(".")[0] for filename in os.listdir(directory) if filename.endswith(extension)]

models = get_models("../models")
print(models)

['gemma-2-9b-it-Q8_0_L', 'saiga-llama-3-q8_0', 'suzume-llama-3-8b-Q8_0']


In [4]:
from langchain_community.llms import LlamaCpp
from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler


llama3_tokens = {
    'first_token': '<|begin_of_text|>',
    'start_header': '<|start_header_id|>', 
    'end_header': '<|end_header_id|>', 
    'end_message': '<|eot_id|>'
}
gemma2_tokens = {
    'first_token': '',
    'start_header': '<start_of_turn>',
    'end_header': '',
    'end_message': '<end_of_turn>'
}

def get_llm(path):

    model = LlamaCpp(
        model_path=path,
        temperature=0,
        n_ctx=8192,
        max_tokens=2000,
        top_p=1,
        repeat_penalty=1.1,
        # callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        verbose=False,
        n_gpu_layers=33
    )
    return model

def get_chat(model_name):
    chat = ChatOllama(model=model_name, temperature=0)
    return chat

In [5]:
extraction_prompt_base = """{first_token}{start_header}system{end_header}

Ты являешься экспертом службы поддержки заполняющим пользовательскую форму для составления заявки.
Пользователь описывает заявку на генерацию текстового контента для социальных сетей.
Твоя задача - заполнить формальную заявку в формате JSON с ключами «Тема», «Социальная сеть», «Целевая аудитория», «Дополнительная информация» исходя из описания пользователя.
Верни JSON без преамбулы или объяснения. Используй язык, на котором пользователь обращается.

Сообщение пользователя для заявки: {user_message}

{end_message}{start_header}assistant{end_header}\n"""

extraction_prompt_candidate_2 = """{first_token}{start_header}system{end_header}

Ты эксперт службы поддержки, заполняющий заявку на генерацию текстового контента для социальных сетей. 
Из описания пользователя заполни формальную заявку в формате JSON с ключами «Тема», «Социальная сеть», «Целевая аудитория», «Дополнительная информация».
Верни JSON без объяснений на языке пользователя.

Сообщение пользователя для заявки: {user_message}

{end_message}{start_header}assistant{end_header}\n"""

extraction_prompt_candidate_3 = """{first_token}{start_header}system{end_header}

Ты эксперт службы поддержки, заполняющий заявку на генерацию текстового контента для социальных сетей.
На основе описания пользователя заполни формальную заявку в формате JSON с ключами: «Тема» (описание основного содержания и целей поста), «Социальная сеть» (указание конкретной платформы), «Целевая аудитория» (детальное описание целевой группы), «Дополнительная информация» (вся прочая важная информация).
Верни JSON без объяснений на языке пользователя.


Сообщение пользователя для заявки: {user_message}

{end_message}{start_header}assistant{end_header}\n"""


extraction_prompt_candidate_4 = """{first_token}{start_header}system{end_header}

Ты эксперт службы поддержки, заполняющий заявку на генерацию текстового контента для социальных сетей. 
На основе описания пользователя заполни формальную заявку в формате JSON с ключами «Тема», «Социальная сеть», «Целевая аудитория», «Дополнительная информация».
Сохраняй язык пользователя. Пример заполненной заявки:

{{
  \"Тема\": \"Проблемы образования в 21 веке\",
  \"Социальная сеть\": \"Instagram\",
  \"Целевая аудитория\": \"Молодежь 15-25 лет решающие куда поступать\",
  \"Дополнительная информация\": \"Использовать хэштеги #образование #университет . Дать советы по выбору образовательного учережденияи его уровня\"
}}

Верни JSON без объяснений.

Сообщение пользователя для заявки: {user_message}

{end_message}{start_header}assistant{end_header}\n"""

extraction_prompt_candidate_5 = """{first_token}{start_header}system{end_header}

Ты эксперт службы поддержки, заполняющий заявку на генерацию текстового контента для социальных сетей. 
На основе описания пользователя заполни формальную заявку в формате JSON с ключами «Тема», «Социальная сеть», «Целевая аудитория», «Дополнительная информация». 
Учитывай особенности каждой социальной сети (например, формат постов в Instagram отличается от формата в Twitter). Разверни все сокращения и аббревиатуры, чтобы обеспечить ясность. 
Сохраняй язык пользователя и верни JSON без объяснений.

Сообщение пользователя для заявки: {user_message}

{end_message}{start_header}assistant{end_header}\n"""

extraction_prompt_candidate_6 = """{first_token}{start_header}system{end_header}

Ты эксперт службы поддержки, заполняющий заявку на генерацию текстового контента для социальных сетей.
На основе описания пользователя заполни формальную заявку в формате JSON с ключами: «Тема» (описание основного содержания и целей поста), «Социальная сеть» (указание конкретной платформы), «Целевая аудитория» (детальное описание целевой группы), «Дополнительная информация» (вся прочая важная информация).
Учитывай особенности каждой социальной сети (например, формат постов в Instagram отличается от формата в Twitter). Разверни все сокращения и аббревиатуры, чтобы обеспечить ясность. 
Верни JSON без объяснений на языке пользователя. Пример заполненной заявки:

{{
  \"Тема\": \"Проблемы образования в 21 веке\",
  \"Социальная сеть\": \"Instagram\",
  \"Целевая аудитория\": \"Молодежь 15-25 лет решающие куда поступать\",
  \"Дополнительная информация\": \"Использовать хэштеги #образование #университет . Дать советы по выбору образовательного учережденияи его уровня\"
}}

Сообщение пользователя для заявки: {user_message}

{end_message}{start_header}assistant{end_header}\n"""


extract_candidates = [
    extraction_prompt_base,
    extraction_prompt_candidate_2,
    extraction_prompt_candidate_3,
    extraction_prompt_candidate_4,
    extraction_prompt_candidate_5,
    extraction_prompt_candidate_6
]

In [6]:
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate

import pandas as pd
import tqdm
from langchain_core.exceptions import OutputParserException


def get_extract_prompt(prompt):
    extract_prompt = PromptTemplate(
        template=prompt,
        input_variables=["first_token", "start_header", "end_header", "end_message", "user_message"],
    )
    return extract_prompt

# extraction_chain = extract_prompt | llm | JsonOutputParser()

In [9]:
results = pd.DataFrame()


for model in get_models("../models"):
    print(model)
    llm = get_chat(model)
    input_dict = gemma2_tokens.copy() if "gemma" in model else llama3_tokens.copy()
    for candidate in tqdm.tqdm(extract_candidates, bar_format='{elapsed}<{remaining}', total=len(extract_candidates)):
        extract_prompt = get_extract_prompt(candidate)
        extraction_chain = extract_prompt | llm | JsonOutputParser()
        outputs = {}
        for num, user_message in enumerate(message_samples):
            input_dict["user_message"] = user_message
            try:
                output = extraction_chain.invoke(input_dict)
            except OutputParserException:
                output = {}
            output = {key: output.get(key, None) for key in ["Тема", "Социальная сеть", "Целевая аудитория", "Дополнительная информация"]}
 
            output["num_message"] = num
            for key in output.keys():
                outputs[key] = outputs.get(key, []) + [output[key]]

        extract_df = pd.DataFrame.from_dict(outputs)
        
        extract_df['stage'] = "extract"
        extract_df['prompt'] = extract_candidates.index(candidate)
        extract_df['model'] = model.split("/")[-1]
        results = pd.concat([results, extract_df], axis=0)

gemma-2-9b-it-Q8_0_L


14:28<00:00


saiga-llama-3-q8_0


13:40<00:00


suzume-llama-3-8b-Q8_0


14:52<00:00


In [7]:
#results.to_csv('./extraction_results.csv', index=False)
results = pd.read_csv('./extraction_results.csv')
results

,Тема,Социальная сеть,Целевая аудитория,Дополнительная информация,num_message,stage,prompt,model
0,Какие книги стоит прочитать каждому?,ВК,"Любители литературы, стремящиеся расширить сво...","Публикация поможет привлечь внимание к книгам,...",0,extract,0,gemma-2-9b-it-Q8_0_L
1,Сколько книг нужно читать в год?,Телеграм,"Люди, интересующиеся интеллектуальным развитие...",Публикация будет содержать личный опыт и совет...,1,extract,0,gemma-2-9b-it-Q8_0_L
2,Топ-5 упражнений для утренней зарядки,Инстаграм,"Люди, стремящиеся начать день с бодрости и эне...",Публикация должна мотивировать аудиторию к зан...,2,extract,0,gemma-2-9b-it-Q8_0_L
3,История успеха известного спортсмена: как прео...,ВКонтакте,"Люди, сталкивающиеся с трудностями в жизни",Рассказ о преодолении препятствий и достижении...,3,extract,0,gemma-2-9b-it-Q8_0_L
4,Как правильно обрезать плодовые деревья весной...,Телеграм,Садоводы и огородники,Публикация поможет поделиться знаниями и опытом,4,extract,0,gemma-2-9b-it-Q8_0_L
...,...,...,...,...,...,...,...,...
715,Советы по созданию эффективной команды,Telegram,"Работники и руководители, интересующиеся вопро...",Использовать формат чата для обмена советами и...,35,extract,5,suzume-llama-3-8b-Q8_0
716,Влияние окружающей среды на здоровье,ВКонтакте,"Люди всех возрастов, интересующиеся вопросами ...",Использовать ссылки на научные исследования и ...,36,extract,5,suzume-llama-3-8b-Q8_0
717,Как развить навыки общения,Telegram,"Люди всех возрастов, заинтересованные в улучше...",Использовать формат чата для обмена советами и...,37,extract,5,suzume-llama-3-8b-Q8_0
718,Влияние путешествий на личность,Twitter,Люди любящие путешествовать и интересующиеся в...,Использовать хэштеги #путешествия #личностноер...,38,extract,5,suzume-llama-3-8b-Q8_0


In [8]:
results.set_index(['prompt', 'model']).groupby(level=['prompt', 'model'])[
    ["Тема", "Социальная сеть", "Целевая аудитория", "Дополнительная информация"]].count() / \
    (results.shape[0] / len(extract_candidates) / len(models))

Тема  Социальная сеть  Целевая аудитория  \
prompt model                                                               
0      gemma-2-9b-it-Q8_0_L    1.000            1.000              1.000   
       saiga-llama-3-q8_0      1.000            1.000              0.975   
       suzume-llama-3-8b-Q8_0  1.000            1.000              1.000   
1      gemma-2-9b-it-Q8_0_L    1.000            1.000              1.000   
       saiga-llama-3-q8_0      1.000            1.000              0.975   
       suzume-llama-3-8b-Q8_0  1.000            1.000              1.000   
2      gemma-2-9b-it-Q8_0_L    1.000            1.000              1.000   
       saiga-llama-3-q8_0      0.925            0.925              0.875   
       suzume-llama-3-8b-Q8_0  1.000            1.000              1.000   
3      gemma-2-9b-it-Q8_0_L    1.000            1.000              1.000   
       saiga-llama-3-q8_0      1.000            1.000              1.000   
       suzume-llama-3-8b-Q8_0  1.000            1.000              1.000   
4      gemma-2-9b-it-Q8_0_L    1.000            1.000              1.000   
       saiga-llama-3-q8_0      1.000            1.000              1.000   
       suzume-llama-3-8b-Q8_0  1.000            1.000              1.000   
5      gemma-2-9b-it-Q8_0_L    1.000            1.000              1.000   
       saiga-llama-3-q8_0      1.000            1.000              1.000   
       suzume-llama-3-8b-Q8_0  1.000            1.000              1.000   

                               Дополнительная информация  
prompt model                                              
0      gemma-2-9b-it-Q8_0_L                        1.000  
       saiga-llama-3-q8_0                          0.700  
       suzume-llama-3-8b-Q8_0                      0.750  
1      gemma-2-9b-it-Q8_0_L                        0.975  
       saiga-llama-3-q8_0                          0.325  
       suzume-llama-3-8b-Q8_0                      0.600  
2      gemma-2-9b-it-Q8_0_L                        0.200  
       saiga-llama-3-q8_0                          0.125  
       suzume-llama-3-8b-Q8_0                      0.875  
3      gemma-2-9b-it-Q8_0_L                        1.000  
       saiga-llama-3-q8_0                          0.650  
       suzume-llama-3-8b-Q8_0                      0.975  
4      gemma-2-9b-it-Q8_0_L                        1.000  
       saiga-llama-3-q8_0                          0.425  
       suzume-llama-3-8b-Q8_0                      0.925  
5      gemma-2-9b-it-Q8_0_L                        0.900  
       saiga-llama-3-q8_0                          1.000  
       suzume-llama-3-8b-Q8_0                      1.000

### Исключаем промпт №2, он может ломать JSON структуру ответа.

In [9]:
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

available_promts = np.delete(results["prompt"].unique(), 2)
results = results[results["prompt"].isin(available_promts)]

In [18]:
test_dataset = pd.DataFrame()
test_dataset["contexts"] = results["num_message"].apply(lambda x: message_samples[int(x)].split("..."))#.astype(str)
test_dataset["answer"] =  "Тема - " + results["Тема"].astype(str) + "Социальная сеть - " + results["Социальная сеть"].astype(str) + \
    "Целевая аудитория - " + results["Целевая аудитория"].astype(str) + "Дополнительная информация - " + results["Дополнительная информация"].astype(str)
test_dataset["question"] = results["num_message"].apply(lambda x: message_samples[int(x)]).astype(str)

In [11]:
import nest_asyncio
nest_asyncio.apply()

In [31]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas import evaluate
from langchain_community.embeddings import OllamaEmbeddings
from datasets import Dataset

metrics = pd.DataFrame(
    columns=["faithfulness"])

critic_model = "gemma-2-27b-it-Q6_K_L"
langchain_llm = ChatOllama(model=critic_model, temperature=0)
langchain_embeddings = OllamaEmbeddings(model=critic_model)

for idx, sample in test_dataset.iterrows():
    

    r = evaluate(
        Dataset.from_pandas(pd.DataFrame(sample).T[['question', 'answer', 'contexts']]),
        metrics=[faithfulness],
        llm=langchain_llm,
        embeddings=langchain_embeddings,
        raise_exceptions=False
    )

    r = dict(r)
    metrics = pd.concat([metrics, pd.DataFrame([r])], axis=0)

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
metrics.to_csv('./extraction_metrics.csv', index=False)
# metrics = pd.read_csv('./extraction_metrics.csv')
# metrics

In [32]:
metrics

,faithfulness
0,NaN
0,NaN
0,1.00
0,1.00
0,1.00
...,...
0,1.00
0,0.60
0,0.60
0,0.75


In [39]:
results = pd.concat([results.reset_index(drop= True), metrics.reset_index(drop= True)], axis=1)

In [146]:
results['prompt'] = results['prompt'] + 1

### Количество выполненных тестов %

In [147]:
results.set_index(['prompt', 'model']).groupby(level=['prompt', 'model'])[
    ["Тема", "Социальная сеть", "Целевая аудитория", "Дополнительная информация", "faithfulness"]].count() / \
    (results.shape[0] / len(available_promts) / len(models))

Тема  Социальная сеть  Целевая аудитория  \
prompt model                                                              
1      gemma-2-9b-it-Q8_0_L     1.0              1.0              1.000   
       saiga-llama-3-q8_0       1.0              1.0              0.975   
       suzume-llama-3-8b-Q8_0   1.0              1.0              1.000   
2      gemma-2-9b-it-Q8_0_L     1.0              1.0              1.000   
       saiga-llama-3-q8_0       1.0              1.0              0.975   
       suzume-llama-3-8b-Q8_0   1.0              1.0              1.000   
4      gemma-2-9b-it-Q8_0_L     1.0              1.0              1.000   
       saiga-llama-3-q8_0       1.0              1.0              1.000   
       suzume-llama-3-8b-Q8_0   1.0              1.0              1.000   
5      gemma-2-9b-it-Q8_0_L     1.0              1.0              1.000   
       saiga-llama-3-q8_0       1.0              1.0              1.000   
       suzume-llama-3-8b-Q8_0   1.0              1.0              1.000   
6      gemma-2-9b-it-Q8_0_L     1.0              1.0              1.000   
       saiga-llama-3-q8_0       1.0              1.0              1.000   
       suzume-llama-3-8b-Q8_0   1.0              1.0              1.000   

                               Дополнительная информация  faithfulness  
prompt model                                                            
1      gemma-2-9b-it-Q8_0_L                        1.000         0.550  
       saiga-llama-3-q8_0                          0.700         0.900  
       suzume-llama-3-8b-Q8_0                      0.750         0.850  
2      gemma-2-9b-it-Q8_0_L                        0.975         0.600  
       saiga-llama-3-q8_0                          0.325         0.875  
       suzume-llama-3-8b-Q8_0                      0.600         0.875  
4      gemma-2-9b-it-Q8_0_L                        1.000         0.725  
       saiga-llama-3-q8_0                          0.650         0.750  
       suzume-llama-3-8b-Q8_0                      0.975         0.800  
5      gemma-2-9b-it-Q8_0_L                        1.000         0.500  
       saiga-llama-3-q8_0                          0.425         0.850  
       suzume-llama-3-8b-Q8_0                      0.925         0.875  
6      gemma-2-9b-it-Q8_0_L                        0.900         0.700  
       saiga-llama-3-q8_0                          1.000         0.850  
       suzume-llama-3-8b-Q8_0                      1.000         0.775

### Количество выполненных тестов %

In [148]:
results.set_index(['model']).groupby(level=['model'])[
    ["faithfulness"]].count() / (results.shape[0] / len(models))

,faithfulness
model,
gemma-2-9b-it-Q8_0_L,0.615
saiga-llama-3-q8_0,0.845
suzume-llama-3-8b-Q8_0,0.835


### Количество выполненных тестов %

In [149]:
results.set_index(['prompt']).groupby(level=['prompt'])[
    ["faithfulness"]].count() / (results.shape[0] / len(models))

,faithfulness
prompt,
1,0.460
2,0.470
4,0.455
5,0.445
6,0.465


### Средняя оценка промпта

In [150]:
results.set_index(['prompt']).groupby(level=['prompt'])[
    ["faithfulness"]].mean() / (results.shape[0] / len(available_promts) / len(models))

,faithfulness
prompt,
1,0.024450
2,0.024078
4,0.022013
5,0.024335
6,0.019983


### Средняя оценка модели

In [151]:
results.set_index(['model']).groupby(level=['model'])[
    ["faithfulness"]].mean() / (results.shape[0] / len(available_promts) / len(models))

,faithfulness
model,
gemma-2-9b-it-Q8_0_L,0.024254
saiga-llama-3-q8_0,0.023059
suzume-llama-3-8b-Q8_0,0.021916


### Средняя оценка

In [152]:
results.set_index(['prompt', 'model']).groupby(level=['prompt', 'model'])[
    ["faithfulness"]].mean() / (results.shape[0] / len(available_promts) / len(models))

faithfulness
prompt model                               
1      gemma-2-9b-it-Q8_0_L        0.025000
       saiga-llama-3-q8_0          0.024687
       suzume-llama-3-8b-Q8_0      0.023844
2      gemma-2-9b-it-Q8_0_L        0.025000
       saiga-llama-3-q8_0          0.023893
       suzume-llama-3-8b-Q8_0      0.023631
4      gemma-2-9b-it-Q8_0_L        0.024765
       saiga-llama-3-q8_0          0.021833
       suzume-llama-3-8b-Q8_0      0.019688
5      gemma-2-9b-it-Q8_0_L        0.024792
       saiga-llama-3-q8_0          0.024853
       suzume-llama-3-8b-Q8_0      0.023571
6      gemma-2-9b-it-Q8_0_L        0.022113
       saiga-llama-3-q8_0          0.019763
       suzume-llama-3-8b-Q8_0      0.018300

### Оценка извлечения соц сети

In [153]:
results.groupby('model')['Социальная сеть'].value_counts(normalize=True)

model                   Социальная сеть
gemma-2-9b-it-Q8_0_L    Телеграм           0.340
                        ВКонтакте          0.210
                        Twitter            0.125
                        Instagram          0.120
                        Telegram           0.085
                        Инстаграм          0.080
                        Твиттер            0.025
                        ВК                 0.015
saiga-llama-3-q8_0      Телеграм           0.235
                        ВКонтакте          0.225
                        Телеграмм          0.165
                        Instagram          0.130
                        Twitter            0.090
                        Инстаграм          0.080
                        Твиттер            0.060
                        Telegram           0.015
suzume-llama-3-8b-Q8_0  ВКонтакте          0.225
                        Instagram          0.215
                        Телеграм           0.210
                        Teleg

In [154]:
results.groupby('prompt')['Социальная сеть'].value_counts(normalize=True)

prompt  Социальная сеть
1       Телеграм           0.425000
        ВКонтакте          0.216667
        Инстаграм          0.133333
        Твиттер            0.091667
        Instagram          0.066667
        Twitter            0.058333
        ВК                 0.008333
2       Телеграм           0.425000
        ВКонтакте          0.216667
        Инстаграм          0.133333
        Twitter            0.100000
        Instagram          0.066667
        Твиттер            0.050000
        ВК                 0.008333
4       ВКонтакте          0.216667
        Instagram          0.208333
        Телеграм           0.200000
        Twitter            0.150000
        Telegram           0.133333
        Телеграмм          0.083333
        ВК                 0.008333
5       ВКонтакте          0.225000
        Телеграм           0.225000
        Instagram          0.200000
        Twitter            0.150000
        Телеграмм          0.133333
        Telegram           0.066667
6   

In [155]:
results["Социальная сеть"].value_counts()

Социальная сеть
Телеграм     157
ВКонтакте    132
Instagram     93
Twitter       73
Telegram      59
Телеграмм     34
Инстаграм     32
Твиттер       17
ВК             3
Name: count, dtype: int64

### Оценка извлечения допольнительной информации

In [156]:
results.groupby('prompt')["Дополнительная информация"].count()


prompt
1     98
2     76
4    105
5     94
6    116
Name: Дополнительная информация, dtype: int64

In [157]:
results.set_index(['prompt', 'model']).groupby(level=['prompt', 'model'])[
    ["Дополнительная информация"]].count()

Дополнительная информация
prompt model                                            
1      gemma-2-9b-it-Q8_0_L                           40
       saiga-llama-3-q8_0                             28
       suzume-llama-3-8b-Q8_0                         30
2      gemma-2-9b-it-Q8_0_L                           39
       saiga-llama-3-q8_0                             13
       suzume-llama-3-8b-Q8_0                         24
4      gemma-2-9b-it-Q8_0_L                           40
       saiga-llama-3-q8_0                             26
       suzume-llama-3-8b-Q8_0                         39
5      gemma-2-9b-it-Q8_0_L                           40
       saiga-llama-3-q8_0                             17
       suzume-llama-3-8b-Q8_0                         37
6      gemma-2-9b-it-Q8_0_L                           36
       saiga-llama-3-q8_0                             40
       suzume-llama-3-8b-Q8_0                         40

In [51]:
results.iloc[0]["Тема"]

'Какие книги стоит прочитать каждому?'

In [56]:
results.iloc[0]["Целевая аудитория"]

'Любители литературы, стремящиеся расширить свой кругозор и найти вдохновение'

In [57]:
results.iloc[0]["Дополнительная информация"]

'Публикация поможет привлечь внимание к книгам, которые могут изменить жизнь читателя.'

In [84]:
results[results["prompt"] == 0].iloc[14]["Дополнительная информация"]

'Публикации должны содержать техники медитации и её преимущества.'

In [85]:
results[results["prompt"] == 5].iloc[14]["Дополнительная информация"]

'Поделиться техниками медитации и её преимуществами'

In [117]:
num_message = 5
prompt = 6

for _, row in results[(results["num_message"] == num_message) & (results["prompt"] == prompt)][["model", "Дополнительная информация"]].iterrows():
    print(row.model, " : ", row["Дополнительная информация"])
    print()

gemma-2-9b-it-Q8_0_L  :  Предложить список сортов томатов, пригодных для сибирского климата. Указать особенности каждого сорта (урожайность, сроки созревания, устойчивость к болезням).

saiga-llama-3-q8_0  :  Использовать хэштеги #томаты #сад #грунт. Дать советы по выбору подходящих сортов и уходу за ними

suzume-llama-3-8b-Q8_0  :  Использовать ссылки на статьи и видео о выращивании томатов. Предложить несколько сортов томатов, подходящих для климата Сибири, с описанием их особенностей и преимуществ.



In [139]:
num_message = 22
prompt = 0

for _, row in results[(results["num_message"] == num_message) & (results["prompt"] == prompt)][["model", "Дополнительная информация"]].iterrows():
    print(row.model, " : ", row["Дополнительная информация"])
    print()

gemma-2-9b-it-Q8_0_L  :  Публикация для обсуждения влияния социальных сетей на психическое состояние

saiga-llama-3-q8_0  :  Обсуждение влияния социальных сетей на наше психическое состояние

suzume-llama-3-8b-Q8_0  :  Обсуждение влияния социальных сетей на психическое состояние



In [134]:
message_samples[22]

'Влияние социальных сетей на психическое здоровье: Эта тема будет актуальна для всех, кто проводит много времени в интернете. Публикация в социальной сети поможет обсудить, как социальные сети влияют на наше психическое состояние. Телеграм'

### Результаты stage extraction  

#### Промпт  
***№ 1.*** Хороший базовый вариант для извлечения информации. Ограничевает модель в формулировках, что приводит к формированию скудных описаний.   Дополнительную информацию для 2-ух из 3-ех моделей генерировал только в 50% случаев.  
***№ 2.*** Вариант с самым простым описанием. Высокие оценки, но проблема с формулировкой информации. Модели полностью переписывают содержание сообщения и плохо извлекают дополнительную информацию.  
***№ 3.*** Был удален, тк у него были прицеденты на повреждение JSON структуры. Это критично и приводило бы к падению агента.  
***№ 4.*** Вариант с примером заполнения JSON показал результат ниже среднего, это было связано с заимствованием информации из примера в тестовых кейсах.  
***№ 5.*** Вариант с описанием ожидаемой информации для JSON ключей. Высокие оценки тестов, но проблема как и с №1 в плохом формировании дополнительной информации.  
***№ 6.*** Вариант включающий все техники. Достаточно высокие оценки по всем моделям при высоком извлечении дополнительной информации. Меньше проблем с формулировками.  

#### Модели  
На текущем этапе появились проблемы с оцениванием модели gemma, возможно, это связано с использованием бОльшей версии как модель оценки.  
Проблемные кейсы были отсмотрены вручную, явных проблем не обнаружено.  
На текущем этапе все модели применимы с той или иной степени, необходимо оценить их поведение на следующих этапах. Наибольший вес будет иметь этап генерации самого поста.